# Cartooning an Image using OpenCV – Python

In [5]:
import cv2

class Cartoonizer:
    """Cartoonizer effect
    A class that applies a cartoon effect to an image.
    The class uses a bilateral filter and adaptive thresholding to create
    a cartoon effect.
    """
    def __init__(self):
        pass

    def render(self, img_path):
        img_rgb = cv2.imread(img_path)

        if img_rgb is None:
            raise FileNotFoundError(f"Image file '{img_path}' not found. Check the path.")

        img_rgb = cv2.resize(img_rgb, (1366, 768))
        numDownSamples = 2  # number of downscaling steps
        numBilateralFilters = 50  # number of bilateral filtering steps

        # -- STEP 1 --
        # Downsample image using Gaussian pyramid
        img_color = img_rgb.copy()
        for _ in range(numDownSamples):
            img_color = cv2.pyrDown(img_color)

        # Apply bilateral filter multiple times
        for _ in range(numBilateralFilters):
            img_color = cv2.bilateralFilter(img_color, 9, 9, 7)

        # Upscale image back to original size
        for _ in range(numDownSamples):
            img_color = cv2.pyrUp(img_color)

        # -- STEPS 2 and 3 --
        # Convert to grayscale and apply median blur
        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
        img_blur = cv2.medianBlur(img_gray, 3)

        # -- STEP 4 --
        # Detect and enhance edges
        img_edge = cv2.adaptiveThreshold(img_blur, 255,
                                         cv2.ADAPTIVE_THRESH_MEAN_C,
                                         cv2.THRESH_BINARY, 9, 2)

        # -- STEP 5 --
        # Convert back to color so that it can be bit-ANDed with the color image
        img_edge = cv2.cvtColor(img_edge, cv2.COLOR_GRAY2BGR)
        img_edge = cv2.resize(img_edge, (img_color.shape[1], img_color.shape[0]))

        return cv2.bitwise_and(img_color, img_edge)

# Initialize the cartoonizer
tmp_canvas = Cartoonizer()

# File name
file_name = "Cartooning_Image/Original.jpg"

# Apply the cartoon effect
res = tmp_canvas.render(file_name)

# Save output
cv2.imwrite("Cartooning_Image/Cartoon_version.jpg", res)

# Display image (for local execution)
cv2.imshow("Cartoonized Image", res)
cv2.waitKey(0)
cv2.destroyAllWindows()


## Simpler implementation

In [7]:
# Import necessary libraries
import cv2
import numpy as np

# Read the image
img = cv2.imread("Cartooning_Image/Original.jpg")  # Ensure the file path is correct

# Check if the image was loaded correctly
if img is None:
    print("Error: Could not load image. Please check the file path.")
else:
    # Convert to grayscale and apply median blur
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.medianBlur(gray, 5)

    # Apply adaptive thresholding to detect edges
    edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                  cv2.THRESH_BINARY, 9, 9)

    # Apply bilateral filter for smooth coloring
    color = cv2.bilateralFilter(img, 9, 250, 250)

    # Combine color and edges to create a cartoon effect
    cartoon = cv2.bitwise_and(color, color, mask=edges)

    # Display images with proper window names
    cv2.imshow("Original Image", img)
    cv2.imshow("Edges", edges)
    cv2.imshow("Cartoon Effect", cartoon)

    # Wait for user input and close windows properly
    cv2.waitKey(0)
    cv2.destroyAllWindows()
